In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

from models import *
from utils import progress_bar

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

In [29]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR100(root='./../data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR100(root='./../data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
print('==> Building model..')
net = ResNet34()

net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

==> Building model..


In [5]:
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc

In [ ]:
start_epoch = 0

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=1e-1, momentum=0.9, weight_decay=5e-4)

In [ ]:
for epoch in range(start_epoch, start_epoch+20):
    train(epoch)
    test(epoch)

In [7]:
# Load checkpoint.
print('==> Resuming from checkpoint..')
assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
checkpoint = torch.load('./checkpoint/ckpt.pth')
net.load_state_dict(checkpoint['net'])
best_acc = checkpoint['acc']

==> Resuming from checkpoint..


### Ortho train

In [8]:
print('==> Building pretrained model..')
net_ortho = ResNet34()

net_ortho = net_ortho.to(device)
if device == 'cuda':
    net_ortho = torch.nn.DataParallel(net_ortho)
    cudnn.benchmark = True

==> Building pretrained model..


In [9]:
# Load checkpoint.
print('==> Resuming from checkpoint..')
assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
checkpoint = torch.load('./checkpoint/ckpt.pth')
net_ortho.load_state_dict(checkpoint['net'])
best_acc = checkpoint['acc']
start_epoch = checkpoint['epoch']

==> Resuming from checkpoint..


In [10]:
num_blocks = [3,4,6,3]

In [11]:
l_imp = {-1:{'conv1':net_ortho.module.bn1.bias.shape[0]}, 0:{}, 1:{}, 2:{}, 3:{}}

mod_id = 0
for module_id in [net_ortho.module.layer1, net_ortho.module.layer2, net_ortho.module.layer3, net_ortho.module.layer4]:
    for b_id in range(num_blocks[mod_id]):
        l_imp[mod_id].update({2*b_id: module_id[b_id].bn1.bias.shape[0]})
        l_imp[mod_id].update({2*b_id+1: module_id[b_id].bn2.bias.shape[0]})
    mod_id += 1
    
normalizer = 0
for key, val in l_imp.items():
    for key1, val1 in val.items():
        normalizer += val1
for key, val in l_imp.items():
    for key1, val1 in val.items():
        l_imp[key][key1] /= normalizer

In [12]:
# Training
def train_ortho(epoch):
    print('\nEpoch: %d' % epoch)
    net_ortho.train()
    running_loss = 0
    correct = 0
    total = 0
    angle_cost = 0.0

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, labels = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net_ortho(inputs)

        L_angle = 0

        ### Conv_ind == 0 ###
        w_mat = net_ortho.module.conv1.weight
        params = (w_mat.reshape(w_mat.shape[0],-1))
        angle_mat = torch.matmul(torch.t(params), params) - torch.eye(params.shape[1]).to(device)
        L_angle += l_imp[-1]['conv1']*(angle_mat).norm(1) #.norm().pow(2))
        
        ### Conv_ind != 0 ###
        mod_id = 0
        for module_id in [net_ortho.module.layer1, net_ortho.module.layer2, net_ortho.module.layer3, net_ortho.module.layer4]:
            for b_id in range(num_blocks[mod_id]):
                w_mat = module_id[b_id].conv1.weight
                params = (w_mat.reshape(w_mat.shape[0],-1))
                angle_mat = torch.matmul(params, torch.t(params)) - torch.eye(params.shape[0]).to(device)
                L_angle += l_imp[mod_id][2*b_id]*(angle_mat).norm(1)

                w_mat = module_id[b_id].conv2.weight
                params = (w_mat.reshape(w_mat.shape[0],-1))
                angle_mat = torch.matmul(params, torch.t(params)) - torch.eye(params.shape[0]).to(device)
                L_angle += l_imp[mod_id][2*b_id+1]*(angle_mat).norm(1)

                try:
                    w_mat = module_id[b_id].shortcut[0]
                    params = (w_mat.reshape(w_mat.shape[0],-1))
                    angle_mat = torch.matmul(params, torch.t(params)) - torch.eye(params.shape[1]).to(device)
                    L_angle += l_imp[mod_id][2*b_id]*(angle_mat).norm(1)
                except:
                    pass
            mod_id += 1
                
        Lc = criterion(outputs, labels)
        loss = (1e-2)*(L_angle) + Lc

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        angle_cost += (L_angle).item()

        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (running_loss/(batch_idx+1), 100.*correct/total, correct, total))

    print("angle_cost: ", angle_cost/batch_idx+1)

In [13]:
def test_ortho(epoch):
    global best_acc_ortho
    net_ortho.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net_ortho(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc_ortho:
        print('Saving..')
        state = {
            'net_ortho': net_ortho.state_dict(),
            'best_acc_ortho': acc
        }
        if not os.path.isdir('ortho_checkpoint'):
            os.mkdir('ortho_checkpoint')
        torch.save(state, './ortho_checkpoint/ortho_ckpt.pth')
        best_acc = acc

In [28]:
def w_diag():
    
    ### Conv_ind == 0 ###
    w_mat = net_ortho.module.conv1.weight
    params = (w_mat.reshape(w_mat.shape[0],-1))
#     print(params.shape)
    angle_mat = torch.matmul(torch.t(params), params) # - torch.eye(params.shape[1]).to(device)
#     print(angle_mat.shape)
    L_diag = (angle_mat.diag().norm(1))
    L_angle = (angle_mat.norm(1))
    print(L_diag.cpu()/L_angle.cpu())

    ### Conv_ind != 0 ###
    mod_id = 0
    for module_id in [net_ortho.module.layer1, net_ortho.module.layer2, net_ortho.module.layer3, net_ortho.module.layer4]:
        for b_id in range(num_blocks[mod_id]):
            w_mat = module_id[b_id].conv1.weight
            params = (w_mat.reshape(w_mat.shape[0],-1))
#             print(params.shape)
            angle_mat = torch.matmul(params, torch.t(params)) # - torch.eye(params.shape[0]).to(device)
#             print(angle_mat.shape)
            L_diag = (angle_mat.diag().norm(1))
            L_angle = (angle_mat.norm(1))
            print(L_diag.cpu()/L_angle.cpu())                

            w_mat = module_id[b_id].conv2.weight
            params = (w_mat.reshape(w_mat.shape[0],-1))
#             print(params.shape)
            angle_mat = torch.matmul(params, torch.t(params)) # - torch.eye(params.shape[0]).to(device)
#             print(angle_mat.shape)
            L_diag = (angle_mat.diag().norm(1))
            L_angle = (angle_mat.norm(1))
            print(L_diag.cpu()/L_angle.cpu())                

            try:
                w_mat = module_id[b_id].shortcut[0]
                params = (w_mat.reshape(w_mat.shape[0],-1))
                angle_mat = torch.matmul(params, torch.t(params)) # - torch.eye(params.shape[1]).to(device)
                L_diag = (angle_mat.diag().norm(1))
                L_angle = (angle_mat.norm(1))
                print(L_diag.cpu()/L_angle.cpu())                
            except:
                pass
        mod_id += 1

In [ ]:
ortho_start = 1

In [ ]:
best_acc_ortho = 0  # best test accuracy

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net_ortho.parameters(), lr=1e-7, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [ ]:
l_imp[-1]['conv1']

In [ ]:
for epoch in range(ortho_start, ortho_start+1):
    train_ortho(epoch)
    test_ortho(epoch)
    w_diag()

In [ ]:
print('==> Resuming from checkpoint..')
assert os.path.isdir('ortho_checkpoint'), 'Error: no checkpoint directory found!'
checkpoint = torch.load('./ortho_checkpoint/ortho_ckpt.pth')
net_ortho.load_state_dict(checkpoint['net_ortho'])
best_acc_ortho = checkpoint['best_acc_ortho']

In [ ]:
test_ortho(epoch)
#w_diag()